<a href="https://colab.research.google.com/github/Hari027/GenAI/blob/main/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math

# 1. PREPARE DATASET
raw_text = """
artificial intelligence is transforming modern society.
it is used in healthcare finance education and transportation.
machine learning allows systems to improve automatically with experience.
data plays a critical role in training intelligent systems.
large datasets help models learn complex patterns.
deep learning uses multi layer neural networks.
neural networks are inspired by biological neurons.
each neuron processes input and produces an output.
training a neural network requires optimization techniques.
gradient descent minimizes the loss function.
natural language processing helps computers understand human language.
text generation is a key task in nlp.
language models predict the next word or character.
recurrent neural networks handle sequential data.
lstm and gru models address long term dependency problems.
however rnn based models are slow for long sequences.
transformer models changed the field of nlp.
they rely on self attention mechanisms.
attention allows the model to focus on relevant context.
transformers process data in parallel.
this makes training faster and more efficient.
modern language models are based on transformers.
education is being improved using artificial intelligence.
intelligent tutoring systems personalize learning.
automated grading saves time for teachers.
online education platforms use recommendation systems.
technology enhances the quality of learning experiences.
ethical considerations are important in artificial intelligence.
fairness transparency and accountability must be ensured.
ai systems should be designed responsibly.
data privacy and security are major concerns.
researchers continue to improve ai safety.
text generation models can create stories poems and articles.
they are used in chatbots virtual assistants and content creation.
generated text should be meaningful and coherent.
evaluation of text generation is challenging.
human judgement is often required.
continuous learning is essential in the field of ai.
research and innovation drive technological progress.
students should build strong foundations in mathematics.
programming skills are important for ai engineers.
practical experimentation enhances understanding.
"""

# Simple Word-Level Tokenizer
tokens = raw_text.lower().replace('.', '').split()
vocab = sorted(list(set(tokens)))
word_to_int = {word: i for i, word in enumerate(vocab)}
int_to_word = {i: word for word, i in word_to_int.items()}
vocab_size = len(vocab)

# Create sequences for training (Input: context, Target: next word)
SEQ_LENGTH = 5
dataX = []
dataY = []
for i in range(len(tokens) - SEQ_LENGTH):
    dataX.append([word_to_int[w] for w in tokens[i:i+SEQ_LENGTH]])
    dataY.append(word_to_int[tokens[i+SEQ_LENGTH]])

X = torch.tensor(dataX)
Y = torch.tensor(dataY)

# 2. POSITIONAL ENCODING
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

# 3. TRANSFORMER MODEL
class TransformerGenerator(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)

        # We use the Encoder as a backbone for features
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=256, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)

        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        x = self.embedding(x) * math.sqrt(128)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        # Take the last word's output for prediction
        out = self.fc_out(x[:, -1, :])
        return out

# 4. TRAINING SETUP
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerGenerator(vocab_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. TRAINING LOOP
print("Starting training...")
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    outputs = model(X.to(device))
    loss = criterion(outputs, Y.to(device))
    loss.backward()
    optimizer.step()
    if (epoch+1) % 50 == 0:
        print(f"Epoch [{epoch+1}/200], Loss: {loss.item():.4f}")

# 6. TEXT GENERATION FUNCTION
def generate_text(model, start_text, length=10):
    model.eval()
    words = start_text.lower().split()
    generated = words[:]

    with torch.no_grad():
        for _ in range(length):
            # Convert last words to indices
            input_indices = [word_to_int.get(w, 0) for w in generated[-SEQ_LENGTH:]]
            input_tensor = torch.tensor([input_indices]).to(device)

            # Predict
            output = model(input_tensor)
            next_word_idx = torch.argmax(output, dim=1).item()
            generated.append(int_to_word[next_word_idx])

    return " ".join(generated)

# 7. OUTPUT SAMPLES
print("\n--- Generated Text Samples ---")
prompt = "artificial intelligence is transforming"
print(f"Prompt: {prompt}")
print(f"Output: {generate_text(model, prompt, length=12)}")

prompt2 = "deep learning uses multi layer"
print(f"\nPrompt: {prompt2}")
print(f"Output: {generate_text(model, prompt2, length=10)}")

Starting training...
Epoch [50/200], Loss: 0.1930
Epoch [100/200], Loss: 0.0387
Epoch [150/200], Loss: 0.0200
Epoch [200/200], Loss: 0.0136

--- Generated Text Samples ---
Prompt: artificial intelligence is transforming
Output: artificial intelligence is transforming and accountability must be ensured ai systems should be designed responsibly data

Prompt: deep learning uses multi layer
Output: deep learning uses multi layer neural networks neural networks are inspired by biological neurons each
